# Teste de Hipóteses - Pense Estatística, 2º Edição.

## Universidade Federal de Alagoas - UFAL
## Centro de Tecnologia - CTEC
## Programa de Pós-Graduação Recursos Hídricos e Saneamento - PPGRHS
### Disciplina de Estatística

Clebson Farias

In [1]:
from __future__ import print_function, division

%matplotlib inline

import numpy as np
import pandas as pd

import thinkstats2
import thinkplot

## Teste de hipóteses classicas

A questão fundamental que queremos abordar é se os efeitos que observamos numa amostra são prováveis de aparecer na população maior.

- **Teoria da decisão:**
    - Teste de hipótese nula de Fisher;
    - Lema de Neyman-Pearson;
    - Inferência Bayesiana.
- Uma hipótese estatística é **uma afirmação sobre os parâmetros** de uma ou mais população;
- Como representamos a população por meio de distribuições, podemos fazer afirmações sobre os seus parâmetros.
- A hipótese envolve um ou mais parâmetros da distribuição;

Passos:
1. Quantificar o efeito aparante sobre a escolha do teste estatístico;
2. Definir as hipótese a ser testada:
    - **Hipótese nula**, é a hipótese que desejamos testar ($H_{0}$);
    - **Hipótese Alternativa**, é a hipótese contrária da hipótese nula ($H_{1}$);
3. Calcular o valor-p, qual a probalidade da hipótese nula ser verdadeira;
4. Interpretar o resultado. Se o valor-p é baixo, o efeito é considerado estatisticamente significante.